In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from ipywidgets import interact
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Preface
Forest fires have become a grave threat for fauna and flora of Amazon rainforests. The agriculture sector has been relentlessly pushing into the rainforests, cutting down trees and burning the forests. All this has been taking place under the nose of the ignorant Brazilian Government even after widespread international protests.
I have done an in-depth analysis of Forest fires in Brazil from 1997 through 2018. The data is obtained from the official website of the Brazilian government.

# Data
The dataset contains monthly count of forest fires in several states of Brazil.<br>
Columns:<br>
year: Year of fire (1998 - 2017)<br>
state: State where fire took place. <br>
    &emsp;&emsp;'Acre',<br> 
    &emsp;&emsp;'Alagoas',<br> &emsp;&emsp;'Amapa',<br> &emsp;&emsp;'Amazonas',<br> &emsp;&emsp;'Bahia',<br>&emsp;&emsp; 'Ceara',<br>
       &emsp;&emsp;'Distrito Federal',<br> &emsp;&emsp;'Espirito Santo',<br> &emsp;&emsp;'Goias',<br> &emsp;&emsp;'Maranhao',<br>
       &emsp;&emsp;'Mato Grosso',<br> &emsp;&emsp;'Minas Gerais',<br> &emsp;&emsp;'Pará',<br> &emsp;&emsp;'Paraiba',<br> &emsp;&emsp;'Pernambuco',<br>
       &emsp;&emsp;'Piau',<br> &emsp;&emsp;'Rio',<br> &emsp;&emsp;'Rondonia',<br> &emsp;&emsp;'Roraima',<br> &emsp;&emsp;'Santa Catarina',<br>
       &emsp;&emsp;'Sao Paulo',<br> &emsp;&emsp;'Sergipe',<br> &emsp;&emsp;'Tocantins'<br>
month: Month of fire. (Jan-Dec)
date: 1st Jan of year of fire. (redundant, hence removed)


In [ ]:
# importing the dataset.
df = pd.read_csv('/kaggle/input/forest-fires-in-brazil/amazon.csv', encoding='latin1')
df = df.drop('date', axis = 1) #removing 'date' column from dataset as it contains the date 1st Jan of fire year.
months = {'Janeiro':'Jan', 'Fevereiro':'Feb', 'Março':'Mar', 'Abril':'Apr', 'Maio':'May', 'Junho':'June', 'Julho': 'July',
       'Agosto':'Aug', 'Setembro':'Sept', 'Outubro':'Oct', 'Novembro': 'Nov', 'Dezembro':'Dec'}
df['month'] = df['month'].apply(lambda x: months[x]) #translating month name to English.
df['number'] = df['number'].round(0) # Rounding off the number of forest fires to integers
df.head()

In [ ]:
# Plotting yearly count trend of fires.
fires = df[['year', 'number']].groupby(['year']).sum()
fires = fires.reset_index()
state_wise = df.groupby(['state', 'year']).sum().reset_index()

In [ ]:
# transforming the data into a matrix with columns representing years and rows containing states' fire counts for corresponding year.
state_wise_yearly = state_wise.pivot(index = 'state', columns = 'year', values = 'number')
state_wise_yearly

# code to highlight maximum fires count each year.
def highlight(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: orange' if v else '' for v in is_max]
state_wise_yearly.style.apply(highlight)

Mato Grosso has seen the maximum number of fires each year since 2001. Before 2001, Sau Paulo used to be the biggest contributor of forest fires owing to its population boom and industrialization.

In [ ]:
state = state_wise[['state', 'number']].groupby(['state']).sum().sort_values(by = ['number'], ascending = False).head(10).reset_index()
plt.figure(figsize = (16,6))
ax = sns.barplot(x = 'state', y = 'number', data = state, palette = 'coolwarm_r')
ax.set(title = 'Statewise count of forest fires (1998 - 2017)', xlabel = 'State', ylabel = 'Forest Fires')

As expected, The biggest contributor of forest fires is Mato Grosso, which lies southwest of the Amazon Rainforests. The dense forests of the state has been cut and burned down for agricultural purposes. Deforestation of the rain forest accelerated significantly with emergence of mechanised agricultural tools and farmers getting into the rich forest lands for soy plantations. <br>
Holding the second place is the coastal state of Paraiba, which has unproportionally large number of fires with respect to its geological area.<br>

States of Paraiba, Sao Paulo, Rio, Bahia, Piau, Goias, and Minas Gerais are not part of Amazon Rainforest area. The 7 of the states with most number of fires are situated towards the east coast. This can be due to increased urbanisation and industrialisation along the coast.

As evident in the satellite photos below, In the period of only 12 years, the large portion of forest cover in Mato Grosso has vanished.<br> 
Source: https://earthobservatory.nasa.gov/images/35891/deforestation-in-mato-grosso-brazil

Satellite Image of Mato Grosso, August 6, 1992.
<img src="https://eoimages.gsfc.nasa.gov/images/imagerecords/35000/35891/matogrosso_l5_1992219_lrg.jpg" alt="drawing" width="400"/>

Satellite Image of Mato Grosso, July 28, 2004.
<img src="https://eoimages.gsfc.nasa.gov/images/imagerecords/35000/35891/matogrosso_ast_2006209_lrg.jpg" alt="drawing" width="400"/>

In [ ]:
#storing longitudinal and latitudinal coordinates for the ten states
state['lat'] = [-16.350000, -22.15847, -23.533773, -22.908333, -11.409874, -21.5089, -16.328547, -19.841644, -21.175, -3.416843]
state['lon'] = [-56.666668, -43.29321, -46.625290, -43.196388, -41.280857, -43.3228, -48.953403, -43.986511, -43.01778, -65.856064]
state["number"] = pd.to_numeric(state["number"]) #casting column values to numeric type

fig = px.scatter_geo(state,  # Input Pandas DataFrame
                    color="state",
                    lat = 'lat',
                    lon = 'lon',
                     size = 'number',# DataFrame column with color values
                    hover_name="state") # displays state name and other details on hovering over the map.
fig.update_layout(
    title_text = 'Number Of Fires', # Create a Title
    geo_scope='south america',  # Plot only South America instead of globe
)
fig.show()

In [ ]:
# interactive graph showing yearly fire count of each state relative to total number of fires.
@interact (state = ['Acre', 'Alagoas', 'Amapa', 'Amazonas', 'Bahia', 'Ceara',
       'Distrito Federal', 'Espirito Santo', 'Goias', 'Maranhao',
       'Mato Grosso', 'Minas Gerais', 'Paraiba', 'Pará', 'Pernambuco',
       'Piau', 'Rio', 'Rondonia', 'Roraima', 'Santa Catarina',
       'Sao Paulo', 'Sergipe', 'Tocantins'])
def plot_graphs(state):
    plt.figure(figsize = (16, 6))
    sns.set_style("whitegrid")
    ax = sns.lineplot(x='year', y='number', data = state_wise[state_wise['state'] == state], color = 'orange', lw = 4, hue = 'state')
    ax = sns.lineplot(x='year', y='number', data = fires, color = 'red', lw = 4, label = 'Total Fires')
    ax.set(xlabel='Year', ylabel='Number of Fires', title = 'Forest Fires in ' + state)
    ax.xaxis.set_major_locator(plt.MaxNLocator(21))
    ax.yaxis.set_major_locator(plt.MaxNLocator(15))
    ax.set_xlim(1998, 2018)
    ax.legend()
    ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
    plt.show()

In [ ]:
# monthly average number of fires during period 1998 - 2017
month_wise = df[['month', 'number']].groupby(['month']).mean().reset_index()
plt.figure(figsize = (16,10))
ax = sns.barplot(x = 'month', y = 'number', data = month_wise, palette = 'magma_r', order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec'])
ax.set(title = 'Average number of forest fires by month', xlabel = 'Number of fires', ylabel = 'Month')

Higher number of forest fires happen in the Amazon during the dry season between July and October. They can be caused by naturally occurring events, like lightning strikes, but most are started by people for industrial and agricultural purposes.

# Conclusion

Forest fires in Brazil don't show any increasing or decreasing trend, but one this sure that not many measures are being taken to curb these. The fires in Amazonas turned the sky dark in city of Sao Paolo which lies 2000 miles east of Amazonas. We can only imagine the damage these fires do to the vegetation and animals living there. Amazon is a very complicated ecosystem which is suffering from deforestation and fires. We need to create awareness about the importance it has for the whole world, not only Brazil. World leaders must come together and care for the forest as if it were their own country.


## END